### This is to contain 
1. General function neeed accross other ipynb files
2. Produce visualization for creating analytics of a given portfolio

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_datareader
import datetime as dt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from IPython.display import display, HTML
pd.options.display.max_columns=200

In [2]:
def historical_exchage_rate_provider_from_yahoo_finance(
    list_of_currencies= ['USD','TWD','AUD','PLN',
                         'CZK','CNY','MXN','CHF',
                         'KRW','QAR','NZD','ILS',
                         'RUB','IDR','TRY','JPY',
                         'CAD','HKD','ZAR','MYR',
                         'BRL','SGD','INR','DKK',
                         'THB','SEK','NOK','GBP',
                         'EUR','CLP']):
    """Get Historical Daily Exchange rate of currencies to USD"""
    ##import yfinance as yf
    ##import pandas as pd
    if len(list_of_currencies)==1:
        df_currency_ex_rate = yf.download([c+"=X" for c in list_of_currencies],
                                          period='max')[['Adj Close']].rename(
            columns={"Adj Close":list_of_currencies[0]+"=X"})
    else:
        df_currency_ex_rate = yf.download([c+"=X" for c in list_of_currencies],
                                          period='max')['Adj Close']
    ### Melt the currency rates
    df_currency_ex_rate = pd.melt(df_currency_ex_rate.reset_index(),
        id_vars='Date',
        value_name='currency_rate',
        var_name='currency')
    ### Drop missing rates
    df_currency_ex_rate.dropna(subset=['currency_rate'],
                               inplace=True)
    ### trim the last 2 character of Currency Name '=X'
    df_currency_ex_rate["currency"]=df_currency_ex_rate["currency"].apply(lambda x: x[:-2])
    ### Fill in missing values of exchange rate wit previous available rate
    df_currency_ex_rate = df_currency_ex_rate.set_index("Date").groupby(
        ['currency']).resample('1D').ffill().drop(['currency'],axis=1).reset_index()
    df_currency_ex_rate = df_currency_ex_rate.sort_values(['currency','Date'])
    df_currency_ex_rate['source']='yahoo'
    return(df_currency_ex_rate)

In [ ]:
def historical_exchage_rate_provider(
    list_of_currencies=['USD','TWD','AUD','PLN',
                         'CZK','CNY','MXN','CHF',
                         'KRW','QAR','NZD','ILS',
                         'RUB','IDR','TRY','JPY',
                         'CAD','HKD','ZAR','MYR',
                         'BRL','SGD','INR','DKK',
                         'THB','SEK','NOK','GBP',
                         'EUR','CLP']):
    """The excel file named currency.xlsx has all the currency and their respective ticker symbol in FRED and Yahoo Finance.
    This functions take those tickers symbols and pull historical currency exchange rate from FED. Then pulls exchange rate from
    Yahoo finance and return aggretated exchange rate from those 2 sources."""
    ##import datetime as dt
    ##import pandas as pd
    ##import pandas_datareader
    ##import yfinance as yf
    ### Get the possible currencies from your data base
    #df_currency = pd.read_excel(path_of_data+"Foreign Exchange Rates/currency.xlsx",sheet_name="currency")
    url_currency = "https://docs.google.com/spreadsheets/d/14t5Yilm6l34AFyb67k3v56R9OpDHZwk8"
    df_currency = pd.read_excel(url_currency+'/export',sheet_name="currency")
    ### Determine which currencies to pull from Fred
    set_of_ticker_currency_to_pull_from_fred = set(df_currency[(df_currency['currency'].isin(list_of_currencies)) &
                                                               (df_currency['source']=='fred')]['ticker'])

    print("Retriving Exchange Rates from FRED")
    df_ex_Rate_fred= pandas_datareader.data.DataReader(set_of_ticker_currency_to_pull_from_fred,
                                                       data_source='fred',
                                                       start=dt.datetime(1950,1,1)).reset_index()
    print("Retriving Exchange Rates from FRED Complete",df_ex_Rate_fred.shape)
    ### Orgonize dataframe
    df_ex_Rate_fred = df_ex_Rate_fred.rename(columns={"DATE":"Date"}).set_index("Date")
    ### Melt the data
    df_ex_Rate_fred = pd.melt(df_ex_Rate_fred.reset_index(),
                                id_vars='Date',
                                value_name='currency_rate',
                                var_name='ticker')
    ### Drop missing rates
    df_ex_Rate_fred.dropna(subset=['currency_rate'],inplace=True)
    ### Resample to have a rate for every possible date in your date range for each currency
    df_ex_Rate_fred = df_ex_Rate_fred.set_index("Date").groupby(['ticker']).resample('1D').ffill().drop(['ticker'],
                                                                                           axis=1).reset_index()
    ### Get the currency standard abbreviation and as well as its target rate
    df_ex_Rate_fred = pd.merge(df_ex_Rate_fred,
                           df_currency[["ticker","toUSD","currency"]],
                           left_on=['ticker'],
                           right_on=['ticker'],
                           how='left')
    ### Some of thses ticker you pulled from the Fred are from 1 Foreign currency to USD.
    ### Most of them are from USD to Foreign Currecny. Make all of them from from USD to Foreign Currecny to keep it consistant with Yahoo Finance
    df_ex_Rate_fred.loc[df_ex_Rate_fred['toUSD']==True,'currency_rate'] = 1.0/df_ex_Rate_fred['currency_rate']
    df_ex_Rate_fred = df_ex_Rate_fred[['currency','Date','currency_rate']]
    df_ex_Rate_fred['source']='fred'

    ####Pull exchange rate from Yhaoo Finance
    print("Retriving Exchange Rates from YahooFinance")
    #print(list_of_currencies)
    df_ex_Rate_yahoo = historical_exchage_rate_provider_from_yahoo_finance(list_of_currencies)
    ### Creata a dataframe for USD to USD exchange rate
    df_ex_rate_usd = pd.DataFrame(pd.date_range(start=dt.datetime(1950,1,1),
                                                end=dt.datetime.now()+ dt.timedelta(days=5),
                                                freq='D'),columns=['Date'])
    df_ex_rate_usd['currency'] = 'USD'
    df_ex_rate_usd['currency_rate'] = 1.00

    ### Concatinate Data from FED and Yahoo Finance and USD Exchange Rate
    df_ex_rate = pd.concat([df_ex_Rate_fred,
                            df_ex_Rate_yahoo,
                            df_ex_rate_usd],
                           ignore_index=True).sort_values(['currency','Date','source'])
    df_ex_rate=df_ex_rate.drop_duplicates(subset=['currency','Date'],
                                          keep='last')
    df_ex_rate.reset_index(drop=True,inplace=True)
    df_ex_rate = df_ex_rate.set_index('Date').groupby(['currency']).resample('1D').ffill().drop(
        ['currency'],axis=1).reset_index()
    df_ex_rate.reset_index(drop=True,inplace=True)
    df_ex_rate.drop(['source'],axis=1,inplace = True)
    return(df_ex_rate)

In [ ]:
def yahoo_quotes_provider_via_yfinance(list_of_symbols):
    """Get Current quote from Yahoo Finance for a list of symbols/tciker"""
    print("Number of symbosl to search on yahoo",len(list_of_symbols))
    list_of_df_quotes=[]
    for symbol in list_of_symbols:
        try:
            symbol_obj = yf.Ticker(symbol)
            symbol_info = symbol_obj.get_info()
            df_quote_yahoo_symbol = pd.DataFrame([symbol_info])
            list_of_df_quotes.append(df_quote_yahoo_symbol)
        except:
            pass
    df_quotes = pd.concat(list_of_df_quotes,
                          ignore_index=True)
    df_quotes.rename(columns={"currency":"symbolCurrency"},inplace=True)
    df_quotes['ebit'] = df_quotes['totalRevenue'] * df_quotes['operatingMargins']
    df_quotes['ev'] = (df_quotes['marketCap'] + df_quotes['totalDebt']) - df_quotes['totalCash'] 
    df_quotes['ev_ebit'] = df_quotes['ev']/df_quotes['ebit']
    df_quotes['netDebt'] = df_quotes['totalDebt']-df_quotes['totalCash']
    df_quotes['netDebt'] = df_quotes['totalDebt']-df_quotes['totalCash']
    df_quotes['industry'] = df_quotes['industry'].fillna("Miscellaneous")
    df_quotes['sector'] = df_quotes['sector'].fillna("Miscellaneous")
    print(len(df_quotes),"quote from Yfinance")
    return(df_quotes)

In [ ]:
def symbol_attribute_retriever_from_yahoo(list_of_symbol):
    ### get histroical exchange rate
    df_ex_rate = historical_exchage_rate_provider()
    df_ex_rate = df_ex_rate[df_ex_rate.groupby(['currency'])['Date'].transform('max') == df_ex_rate['Date']].reset_index(drop=True)
    
    df_place_holder = pd.DataFrame(columns=['symbol','exchange','shortName','industry','sector','heldPercentInsiders','currentPrice','bookValue',
                                        'returnOnAssets','returnOnEquity','firm_to_ebit','priceToBook','trailingPE','forwardPE','dividendYield',
                                            'payoutRatio','fiveYearAvgDividendYield','quickRatio', 'currentRatio','debtToEquity',
                                        'grossMargins','ebitdaMargins','operatingMargins','profitMargins',
                                        'marketCap','ebit','netIncomeToCommon','totalCash','netDebt','totalDebt','totalRevenue','grossProfits','ebitda','operatingCashflow','freeCashflow',
                                        'symbolCurrency','financialCurrency','fullTimeEmployees','longBusinessSummary'])
    df_symbol_attributes_from_yahoo = yahoo_quotes_provider_via_yfinance(list_of_symbol)
    ### pull fudamenetal from yfinance
    df_symbol_attributes_from_yahoo = pd.concat([df_place_holder,df_symbol_attributes_from_yahoo],ignore_index=True)
    ### mereg merge rate with attribuetfb rom yfiance
    df_symbol_attributes_from_yahoo = pd.merge(df_symbol_attributes_from_yahoo,
         df_ex_rate[['currency','currency_rate']].rename(
             columns={"currency":"symbolCurrency",
                      "currency_rate":"currency_rate_symbol"}),
         left_on=['symbolCurrency'],
         right_on=['symbolCurrency'],
         how='left')
    df_symbol_attributes_from_yahoo = pd.merge(df_symbol_attributes_from_yahoo,
             df_ex_rate[['currency','currency_rate']].rename(columns={"currency":"financialCurrency","currency_rate":"currency_rate_financial"}),
             left_on=['financialCurrency'],
             right_on=['financialCurrency'],
             how='left')

    list_of_col_to_convert_to_usd_symbol = ['currentPrice','marketCap']
    list_of_col_to_convert_to_usd_financial = ['totalRevenue','grossProfits','ebitda','ebit',
                                               'netIncomeToCommon','operatingCashflow','bookValue',
                                               'freeCashflow','totalCash','totalDebt','netDebt']
    ### Make a copy of what you pulled from Yfinanc and convert them to USD
    df_symbol_attributes_from_yahoo_usd = df_symbol_attributes_from_yahoo.copy()
    df_symbol_attributes_from_yahoo_usd[list_of_col_to_convert_to_usd_financial] = df_symbol_attributes_from_yahoo_usd[list_of_col_to_convert_to_usd_financial].divide(
        df_symbol_attributes_from_yahoo_usd['currency_rate_financial'],axis='index')

    df_symbol_attributes_from_yahoo_usd[list_of_col_to_convert_to_usd_symbol] = df_symbol_attributes_from_yahoo_usd[list_of_col_to_convert_to_usd_symbol].divide(
        df_symbol_attributes_from_yahoo_usd['currency_rate_symbol'],axis='index')
    
    df_symbol_attributes_from_yahoo_usd['firm_to_ebit'] = (df_symbol_attributes_from_yahoo_usd['marketCap']+df_symbol_attributes_from_yahoo_usd['totalDebt'])/ df_symbol_attributes_from_yahoo_usd['ebit']
    ### covnert ratio inn %
    for col in ['returnOnAssets','returnOnEquity','heldPercentInsiders','grossMargins','ebitdaMargins','operatingMargins','profitMargins','dividendYield','payoutRatio']:
        df_symbol_attributes_from_yahoo_usd[col]= df_symbol_attributes_from_yahoo_usd[col]*100
        
    ## convert numebr ijnto MM 
    for col in ['ev','marketCap','totalRevenue','grossProfits','ebitda','ebit','netIncomeToCommon',
                'operatingCashflow','freeCashflow','totalCash','totalDebt','netDebt']:
        df_symbol_attributes_from_yahoo_usd[col]= df_symbol_attributes_from_yahoo_usd[col]/1000000
    
    df_symbol_attributes_from_yahoo_usd = df_symbol_attributes_from_yahoo_usd[
        ['symbol','shortName','industry','sector','heldPercentInsiders','currentPrice','bookValue',
         'returnOnAssets','returnOnEquity','ev','ev_ebit','firm_to_ebit','priceToBook','trailingPE','forwardPE',
         'dividendYield','payoutRatio','fiveYearAvgDividendYield','quickRatio', 'currentRatio','debtToEquity',
         'grossMargins','ebitdaMargins','operatingMargins','profitMargins',
         'marketCap','ebit','netIncomeToCommon','netDebt','totalCash','totalDebt','totalRevenue','grossProfits','ebitda','operatingCashflow','freeCashflow',
         'symbolCurrency','financialCurrency','exchange','fullTimeEmployees','longBusinessSummary']].copy()
    return(df_symbol_attributes_from_yahoo_usd)

In [ ]:
def data_frame_flattener(df_data):
    df=df_data.copy()
    try:
        df.columns=[' '.join(map(str,col)).strip() for col in df.columns.values]
    except:
        pass
    return(df)

def column_suffix_adder(df_data,
                        list_of_columns_to_add_suffix_on,
                        suffix):
    """Add specific siffix to specific columns"""
    df=df_data.copy()
    ### Add suffix or prefix to certain columns rename all columns
    new_names = [(i,i+suffix) for i in df[list_of_columns_to_add_suffix_on].columns.values]
    df.rename(columns = dict(new_names), inplace=True)
    return(df)

### Visualization 

In [ ]:
def plotly_line_dash_bar_chart(df_data,
                               x,
                               ybar_left=[],
                               ybar_right=[],
                               yline_left=[],
                               yline_right=[],
                               ydash_left=[],
                               ydash_right=[],
                               height=500, 
                               width=1600,
                               rangemode=None,
                               title=None,
                               barmode='group',
                               texttemplate= "%{value}"
                               ):
    """ Graph 2 time series on 2 different y-axis"""
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    #fig.update_yaxes(rangemode='tozero')

    for bar_var in ybar_left:
        # Add traces
        fig.add_trace(
            go.Bar(x=df_data[x],
                   y=df_data[bar_var],
                   name=bar_var,
                   #text = df_data[bar_var],
                   textposition="inside",
                   texttemplate= texttemplate,
                   textfont_color="white"),
            secondary_y=False,
            )
    for bar_var in ybar_right:
        # Add traces
        fig.add_trace(
            go.Bar(x=df_data[x],
                   y=df_data[bar_var],
                   name=bar_var,
                   #text = df_data[bar_var],
                   textposition="inside",
                   texttemplate= texttemplate,
                   textfont_color="white"),
            secondary_y=True,
            )
    
    for line_var in yline_left:
        fig.add_trace(
            go.Scatter(x=df_data[x], 
                       y=df_data[line_var],
                       name=line_var,
                       #hoverinfo='text',
                       ),
            secondary_y=False,
            )
    
    for line_var in yline_right:
        fig.add_trace(
            go.Scatter(x=df_data[x], 
                       y=df_data[line_var],
                       name=line_var
                       ),
            secondary_y=True,
            )
        
    for dash_var in ydash_left:
        fig.add_trace(
            go.Scatter(x=df_data[x],
                       y=df_data[dash_var],
                       name=dash_var,
                       line = dict(dash='dot')),
            secondary_y=False,
            )
    for dash_var in ydash_right:
        fig.add_trace(
            go.Scatter(x=df_data[x],
                       y=df_data[dash_var],
                       name=dash_var,
                       line = dict(dash='dot')),
            secondary_y=True,
            )
    if rangemode != None:
        fig.update_yaxes(rangemode=rangemode)
    fig = fig.update_layout(height=height, 
                            width=width,
                            title=title,
                            barmode=barmode)
    return(fig)

In [ ]:
def net_trade_plotter(df_data,
                      height=530,
                      width=1600):
    df_trade_summary = df_data.copy()
    fig = plotly_line_dash_bar_chart(df_trade_summary,
                               x='date',
                                ybar_left=['trade_amount buy','trade_amount sell'],
                                ybar_right=[],
                                yline_left=[],
                                yline_right=['Net'],
                                ydash_left=[],
                                ydash_right=[],
                                height=530,
                                width=1600,
                                rangemode=None,
                                title="Inflow - Outflow",
                                barmode='relative',
                                texttemplate='%{y:,.0f}',
                                     #texttemplate= "%{value}"
                                    ).update_layout(hovermode='x')

    left_min = pd.melt(df_trade_summary,id_vars='date',value_vars=['trade_amount buy','trade_amount sell'])['value'].min()*1.1
    left_max = pd.melt(df_trade_summary,id_vars='date',value_vars=['trade_amount buy','trade_amount sell'])['value'].max()*1.1

    right_min = pd.melt(df_trade_summary,id_vars='date',value_vars=['Net'])['value'].min()*1.1
    right_max =pd.melt(df_trade_summary,id_vars='date',value_vars=['Net'])['value'].max()*1.1
    

    fig.update_layout(yaxis=dict(range=[left_min,left_max]), 
                      yaxis2=dict(range=[left_min,left_max]),
                      legend=dict(
                          yanchor="top",
                          y=1.14,
                          xanchor="left",
                          x=0))
    fig.update_yaxes(rangemode ='tozero')
    return(fig)

In [ ]:
def portfolio_analytics(res_my_portfolio=None,
                        res_trade_histroy=None,
                        height = 530,
                        width = 1600):
    #### Pull Data Current Holdings
    if res_my_portfolio == None:
        print("Pulling Portfolio Data")
        res_my_portfolio = my_portfolio()
    if res_trade_histroy == None:
        print("Pulling trade history")
        res_trade_histroy = trade_histroy()
    df_my_current_holding = res_my_portfolio['my_current_holdings'].copy().reset_index().rename(columns={'index':'rank'})
    df_trade_summary = res_trade_histroy['df_trade_summary'].copy()
    df_open_orders = res_my_portfolio['open_orders'].copy()
    df_portfolio_hist = res_my_portfolio['portfolio_history'].copy()
    ### Add some columns you need to chart creatation and delete them afterwards
    df_my_current_holding['rank'] = df_my_current_holding['rank']+1
    df_my_current_holding['symbol_rank'] = df_my_current_holding['symbol'].astype(str)+" , " +df_my_current_holding['rank'].astype(str)
    df_my_current_holding['name_sector'] = df_my_current_holding['sector'] + " - " + df_my_current_holding['name']
    df_my_current_holding['name_industry'] = df_my_current_holding['industry'] + " - " + df_my_current_holding['name']
    
    for col in ['trade_amount buy', 'trade_amount sell', 'Net']:
        df_trade_summary[col] = np.round(df_trade_summary[col],0)
    ### Inflow outflow
    fig_inflow_outflow = net_trade_plotter(df_trade_summary.set_index("date").resample("1m").sum().reset_index()[2:],
                                           height=height,
                                           width=width)
    ### Timeseries of portfolio
    fig_portfolio_timeseries = plotly_line_dash_bar_chart(
        df_portfolio_hist[df_portfolio_hist['begins_at']>dt.datetime(2020,8,15)],
                                   x='begins_at',
                                   ybar_left=[],
                                   ybar_right=[],
                                   yline_left=['adjusted_close_equity'],
                                   yline_right=[],
                                   ydash_left=[],
                                   ydash_right=['CumReturn'],
                                   height=height, 
                                   width=width,
                                   rangemode=None,
                                   title=None,
                                   barmode='group',
                                   texttemplate= "%{value}"
                                   )
    
    fig_portfolio_timeseries.update_layout(hovermode="x",
                                           legend=dict(
                                               yanchor="top",
                                               y=1.2,
                                               xanchor="left",
                                               x=0))
    display(df_portfolio_hist[-7:])
    fig_portfolio_timeseries.show()
    ### pie charts
    fig_inflow_outflow.show()
    ### Sector table
    print("Sector | Holding Summary")
    df_my_current_holding['All'] = 'All'
    df_my_current_holding_sector_summary = pd.concat([holding_summerizer(df_my_current_holding,
                                                                          ['sector']),
                                                       holding_summerizer(df_my_current_holding,
                                                                          ['All']).rename(
                                                           columns={"All":"sector"})],
                                                      ignore_index=True).sort_values(
        "equity",ascending=False).reset_index(drop=True)
    
    
    
    df_my_current_holding.drop(['All'],axis=1,inplace=True)
    display(df_my_current_holding_sector_summary)
    ###### Sector Bar chart
    fig_sector_bar = px.bar(pd.merge(df_my_current_holding,
             df_my_current_holding.groupby(['sector']).agg(equity_per_sector= ('equity','sum')).reset_index(),
             left_on=['sector'],
             right_on=['sector'],
             how='left').sort_values(['equity_per_sector','equity'],ascending=[False,False]),
                 x="sector", 
                 y="equity", 
                 color="name_sector", 
                 text="symbol",
                 title='Long Short Sector | Investment Excl. Cash',
          hover_data={'equity':':,.0f','equity_change':':,.0f','percent_change':':,.1f','percentage':':,.1f','ROE':':,.1f'},
           text_auto='$.2s',
          hover_name='name')
    dfs = df_my_current_holding.groupby(['sector']).agg(total_equity= ('equity','sum')).reset_index()
    fig_sector_bar.add_trace(go.Scatter(
        x=dfs['sector'], 
        y=dfs['total_equity'],
        #text=dfs['equity'],
        mode='markers',
        marker_size = 9,
        texttemplate= '%{y:,.0f}',
        showlegend=False
    ))
    fig_sector_bar.update_layout(height=height, width=width)
    fig_sector_bar.update_yaxes(tickformat=".2s")
    fig_sector_bar.show()

    
    ### Sector pie chart
    fig_pie_holding = px.sunburst(df_my_current_holding[df_my_current_holding['equity']>0],
                                  path=['sector', 'symbol'], values='equity',
                                    hover_data={'industry':True,'name':True,'rank':':.0f','percent_change':':.1f','equity_change':':,.0f'},
                                  height=690,
                                  title='Long Sector | Investment Excl. Cash')
    fig_pie_holding.update_traces(texttemplate = '%{label}<br>%{percentRoot:.1%}')
    fig_pie_holding.show()
    ### Industry bar chart
    fig_industry_bar = px.bar(pd.merge(df_my_current_holding,
             df_my_current_holding.groupby(['industry']).agg(equity_per_industry= ('equity','sum')).reset_index(),
             left_on=['industry'],
             right_on=['industry'],
             how='left').sort_values(['equity_per_industry','equity'],ascending=[False,False]),
                 x="industry", 
                 y="equity", 
                 color="name_industry", 
                 text="symbol",
                 title='Long Short industry | Investment Excl. Cash',
          hover_data={'equity':':,.0f','equity_change':':,.0f','percent_change':':,.1f','percentage':':,.1f','ROE':':,.1f'},
           text_auto='$.2s',
           #barmode='relative',
          hover_name='name')
    dfs = df_my_current_holding.groupby(['industry']).agg(total_equity= ('equity','sum')).reset_index()
    fig_industry_bar.add_trace(go.Scatter(
        x=dfs['industry'], 
        y=dfs['total_equity'],
        mode='markers',
        marker_size = 7,
        texttemplate= '%{y:,.0f}',
        showlegend=False
    ))
    fig_industry_bar.update_layout(height=height, width=width,
                                   xaxis={"range":[-.5,len(dfs)+.2]})
    fig_industry_bar.update_yaxes(tickformat=".2s")
    fig_industry_bar.show()

    
    
    ### Industry pie charts
    fig_pie_holding3 = px.sunburst(df_my_current_holding[df_my_current_holding['equity']>0],
                                   path=['industry', 'symbol'],
                                   values='equity',
                                   hover_data= {'sector':True,'name':True,'rank':':.0f','percent_change':':.1f','equity_change':':,.0f'},
                height=690,
                                  title='Long Industry | Investment Excl. Cash')
    fig_pie_holding3.update_traces(texttemplate = '%{label}<br>%{percentRoot:.1%}')
    fig_pie_holding3.show()
    
    ### individual Holdings pie chart
    fig_pie_holding_2 = px.sunburst(df_my_current_holding,
                                  path=['symbol'], values='equity',
                hover_data=['sector','industry','percent_change','name','rank'],
                height=690,
                                  title='Long Holdings | Investment Excl. Cash')
    fig_pie_holding_2.update_traces(texttemplate = '%{label}<br>%{percentRoot:.1%}')
    fig_pie_holding_2.show()
    
    ### paretor distribution 
    fig_pareto_holding_ = plotly_line_dash_bar_chart(df_my_current_holding,
                               x='symbol_rank',
                               ybar_left=['percentage'],
                               ybar_right=[],
                               yline_left=[],
                               yline_right=['CumPercent'],
                               ydash_left=[],
                               ydash_right=[],
                               height=height*.9, 
                               width=width*.9,
                               rangemode=None,
                               title='Pareto Distribution of Holdings',
                               barmode='group',
                               texttemplate= "%{value}").update_layout(hovermode="x",
                                                                       legend=dict(
                                                                       yanchor="top",
                                                                       y=1.16,
                                                                       xanchor="left",
                                                                       x=0))
    fig_pareto_holding_.show()
    df_my_current_holding['fpe'] = df_my_current_holding['fpe'].astype(float)
    df_my_current_holding = df_my_current_holding.drop(['symbol_rank'],axis=1).copy()

    
    ### Drop the unwaneted columns
    df_my_current_holding.drop(['name_sector','name_industry'],axis=1,inplace=True)
    ### Freez 1st row
    df_my_current_holding_formatted = df_my_current_holding.style.set_sticky(axis=1)
    ### format numbers
    df_my_current_holding_formatted = df_my_current_holding_formatted.format({"ROA": "{:,.2f}",
                                                                                "ROE": "{:,.2f}",
                                                                                "quantity": "{:,.0f}",
                                                                                "equity": "{:,.0f}",
                                                                                "equity_initial": "{:,.0f}",
                                                                                "equity_change": "{:,.0f}",
                                                                                'percent_change': "{:,.2f}",
                                                                                'ave_price': "{:,.2f}",
                                                                                 "price": "{:,.2f}",
                                                                                  'ev': "{:,.2f}",
                                                                                'ev_ebit': "{:,.2f}",
                                                                                "firm_to_ebit": "{:,.2f}",
                                                                                "pe": "{:,.2f}",
                                                                                "fpe": "{:,.2f}",
                                                                                "pb": "{:,.2f}",
                                                                                "pb_ratio": "{:,.2f}",
                                                                              "percentage": "{:,.2f}",
                                                                              "CumPercent": "{:,.2f}",
                                                                              "ev": "{:,.0f}",  
                                                                              "marketCap": "{:,.0f}",
                                                                                "rev": "{:,.0f}",
                                                                                "ebit": "{:,.0f}",
                                                                              "netIncome":"{:,.0f}",
                                                                              "netDebt":"{:,.0f}",
                                                                                "grossMargins": "{:,.2f}",
                                                                                "operatingMargins": "{:,.2f}",
                                                                                "profitMargins": "{:,.2f}",
                                                                                "totalCash": "{:,.0f}",
                                                                                "totalDebt": "{:,.0f}",
                                                                                "currentRatio": "{:,.2f}",
                                                                                "quickRatio": "{:,.2f}",
                                                                                "grossProfits": "{:,.2f}",
                                                                                "ebitda": "{:,.0f}",
                                                                                "ocf": "{:,.0f}",
                                                                                "ebitdaMargins": "{:,.2f}",
                                                                                "freeCashflow": "{:,.0f}",
                                                                                "heldPercentInsiders": "{:,.2f}",
                                                                                "div_yeild": "{:,.2f}",
                                                                                 "5y_div_yeild": "{:,.2f}",
                                                                                 "payoutRatio": "{:,.2f}",
                                                                                "fullTimeEmployees": "{:,.0f}",
                                                                                'debtToEquity': "{:,.2f}"})
    ### Color Code columns
    from  matplotlib.colors import LinearSegmentedColormap
    c = ["darkred","red","lightcoral","white", "palegreen","green","darkgreen"]
    v = [0,.15,.4,.5,0.6,.9,1.]
    l = list(zip(v,c))
    cmap=LinearSegmentedColormap.from_list('rg',l, N=256)
    
    c2 = ["darkred","lightcoral","white","white","white","lightcoral","darkred"]
    v2 = [0,.2,.4,.5,0.6,.8,1.]
    l2 = list(zip(v2,c2))
    cmap2 =LinearSegmentedColormap.from_list('range',l2, N=256)
    
    df_my_current_holding_formatted.background_gradient(cmap=cmap2, subset=['percentage'])
    df_my_current_holding_formatted.background_gradient(vmin=-100,vmax=100,cmap=cmap, subset=['percent_change'])
    df_my_current_holding_formatted.background_gradient(cmap=cmap, subset=['equity_change'])
    df_my_current_holding_formatted.background_gradient(cmap='Blues', subset=['equity'])
    #df_my_current_holding_formatted.background_gradient(cmap='Blues', subset=['percentage'])
    df_my_current_holding_formatted.background_gradient(cmap='Blues', subset=['CumPercent'])
    display(df_my_current_holding_formatted)
    print("Open Orders")
    display(df_open_orders)

In [ ]:
def portfolio_consolidator(dict_portfolio_robinhood,
                           dict_trade_histroy_robinhood,
                           dict_portfolio_ibrk,
                           dict_trade_histroy_ibrk):
    dict_my_portfolio_robinhood = dict_portfolio_robinhood.copy()
    dict_trade_histroy_robinhood = dict_trade_histroy_robinhood.copy()
    dict_my_portfolio_ibkr = dict_portfolio_ibrk.copy()
    dict_trade_histroy_ibkr = dict_trade_histroy_ibrk.copy()
    ### 1. Portfolio history
    df_portfolio_history_robinhood = dict_my_portfolio_robinhood['portfolio_history']
    df_portfolio_history_robinhood['broker'] = 'robinhood'
    df_portfolio_history_ibkr = dict_my_portfolio_ibkr['portfolio_history']
    df_portfolio_history_ibkr['broker'] = 'ibkr'

    ### resample ibkr values back to the 1st date that you have in robinhood account.
    ### We know robinhood give back data up to 5 years ago. It will be problem down the read in 2 years from now
    ### Will have to find a way to resolve that then. 
    min_date_in_robinhood = df_portfolio_history_robinhood['begins_at'].min()

    df_portfolio_history_ibkr_to_append = df_portfolio_history_ibkr[df_portfolio_history_ibkr['begins_at'] == df_portfolio_history_ibkr['begins_at'].min()].copy()
    df_portfolio_history_ibkr_to_append['begins_at'] = min_date_in_robinhood
    for col in ['Stocks', 'Cash','close_market_value', 'Net Deposit', 'Cum Net Deposit', 'CumPnL', 'PnL','Return', 'CumReturn']:
        df_portfolio_history_ibkr_to_append[col] = 0
    df_portfolio_history_ibkr  = pd.concat([df_portfolio_history_ibkr,df_portfolio_history_ibkr_to_append],ignore_index=True)
    df_portfolio_history_ibkr = df_portfolio_history_ibkr.sort_values("begins_at").reset_index(drop=True)
    ### You may want to use a if logic to the use the NAV via the rest API. 
    ### That could work if no new depost or widreal has been made
    df_portfolio_history_all = pd.concat([df_portfolio_history_robinhood,
                                      df_portfolio_history_ibkr],ignore_index=True)
    # ### resample for holidays
    df_portfolio_history_all = df_portfolio_history_all.set_index("begins_at").groupby(['broker']).resample('1D').apply(
                                                                                                                        {'adjusted_close_equity': 'ffill',
                                                                                                                         'close_market_value':'ffill',
                                                                                                                         'CumPnL':'ffill',
                                                                                                                         'PnL':'mean'}).reset_index()

    ### Create Portfolio histroy surramry accross all accounts
    df_portfolio_history = df_portfolio_history_all.groupby(['begins_at'])[['close_market_value','adjusted_close_equity','PnL']].sum().reset_index()
    df_portfolio_history['CumPnL'] =  df_portfolio_history['PnL'].cumsum()
    df_portfolio_history['Return']  = df_portfolio_history['adjusted_close_equity'].pct_change().fillna(0)
    df_portfolio_history['CumReturn'] = ((df_portfolio_history['Return']+1).cumprod()-1)*100
    df_portfolio_history['Return'] = df_portfolio_history['Return'] *100
    
    ####### 2. current holding
    df_my_current_holding_robinhood = dict_my_portfolio_robinhood['my_current_holdings']
    df_my_current_holding_robinhood['broker'] = 'robinhood'
    df_my_current_holding_ibkr = dict_my_portfolio_ibkr['my_current_holdings']
    df_my_current_holding_ibkr['broker'] = 'ibkr'
    df_my_current_holding_all = pd.concat([df_my_current_holding_robinhood,
                                           df_my_current_holding_ibkr],
                                          ignore_index=True)
    df_my_current_holding = df_my_current_holding_all.groupby(
        ['symbol','name','sector','industry','exchange',
         'symbolCurrency','financialCurrency'],
        dropna=False).apply(lambda s: 
                                                        pd.Series({ 
                                                            "ROA": s['ROA'].mean(),
                                                            "ROE": s['ROE'].mean(),
                                                            "quantity": s["quantity"].sum(),
                                                            "equity": s["equity"].sum(),
                                                            "equity_initial": s["equity_initial"].sum(),
                                                            "equity_change": s["equity_change"].sum(),
                                                            'percent_change': np.average(a=s['percent_change'],weights=s['equity_initial']),
                                                            'ave_price': np.average(a=s['ave_price'],weights=s['quantity']),
                                                            "price": s['price'].mean(),
                                                            'ev_ebit':s['ev_ebit'].mean(),
                                                            "firm_to_ebit": s['firm_to_ebit'].mean(),
                                                            "pe": s['pe'].mean(),
                                                            "fpe": s['fpe'].mean(),
                                                            "pb": s['pb'].mean(),
                                                            "pb_ratio": s['pb_ratio'].mean(),
                                                            "ev": s['ev'].mean(),
                                                            "marketCap": s['marketCap'].mean(),
                                                            "rev": s['rev'].mean(),
                                                            "ebit": s['ebit'].mean(),
                                                            "netIncome": s['netIncome'].mean(),
                                                            "netDebt": s['netDebt'].mean(),
                                                            "grossMargins": s['grossMargins'].mean(),
                                                            "operatingMargins": s['operatingMargins'].mean(),
                                                            "profitMargins": s['profitMargins'].mean(),
                                                            "totalCash": s['totalCash'].mean(),
                                                            "totalDebt": s['totalDebt'].mean(),
                                                            "currentRatio": s['currentRatio'].mean(),
                                                            "quickRatio": s['quickRatio'].mean(),
                                                            "grossProfits": s['grossProfits'].mean(),
                                                            "ebitda": s['ebitda'].mean(),
                                                            "ocf": s['ocf'].mean(),
                                                            "ebitdaMargins": s['ebitdaMargins'].mean(),
                                                            "freeCashflow": s['freeCashflow'].mean(),
                                                            "heldPercentInsiders": s['heldPercentInsiders'].mean(),
                                                            "div_yeild": s['div_yeild'].mean(),
                                                             "5y_div_yeild": s['5y_div_yeild'].mean(),
                                                             "payoutRatio": s['payoutRatio'].mean(),
                                                            "fullTimeEmployees": s['fullTimeEmployees'].mean(),
                                                            'debtToEquity': s['debtToEquity'].mean(),
                                                            'boker_equity_K': dict(zip(s['broker'],np.round(s['equity']/1000,2)))
                                                        })).sort_values("equity",ascending=False).reset_index()

    ### calcualte % of potoflio
    df_my_current_holding['percentage'] = (df_my_current_holding['equity']/df_my_current_holding['equity'].sum())*100
    df_my_current_holding['CumPercent'] = df_my_current_holding['percentage'].cumsum()
    ### move columsna around
    df_my_current_holding = df_my_current_holding[['symbol', 'name', 'sector', 'industry', 'ROA', 'ROE', 'quantity', 'equity',
                                                   'equity_initial', 'equity_change', 'percent_change','percentage', 'CumPercent', 'ave_price',
                                                   'price', 'ev_ebit', 'firm_to_ebit', 'pe', 'fpe', 'pb', 'pb_ratio', 'ev',
                                                   'marketCap', 'rev', 'ebit', 'netIncome', 'netDebt', 'grossMargins',
                                                   'operatingMargins', 'profitMargins', 'totalCash', 'totalDebt',
                                                   'currentRatio', 'quickRatio', 'grossProfits', 'ebitda', 'ocf',
                                                   'ebitdaMargins', 'freeCashflow', 'heldPercentInsiders', 'div_yeild',
                                                   '5y_div_yeild', 'payoutRatio', 'fullTimeEmployees', 'debtToEquity',
                                                   'boker_equity_K','exchange', 'symbolCurrency',
                                                   'financialCurrency']].copy()

    ### 3. Trade 
    df_trade_summary_robin = dict_trade_histroy_robinhood['df_trade_summary']
    df_trade_summary_robin['broker'] = 'robinhood'
    df_trade_summary_ibkr = dict_trade_histroy_ibkr['df_trade_summary']
    df_trade_summary_ibkr['broker'] = 'ibkr'

    df_trade_summary = pd.concat([df_trade_summary_robin,df_trade_summary_ibkr],ignore_index=True)
    df_trade_summary = df_trade_summary.groupby(['date'])[['trade_amount buy','trade_amount sell','Net']].sum().reset_index()
    
    ### open order 
    df_open_orders_robin = dict_my_portfolio_robinhood['open_orders']
    df_open_orders_robin['broker'] = 'robinhood'
    df_open_orders_ibkr = dict_my_portfolio_ibkr['open_orders']
    df_open_orders_ibkr['broker'] = 'ibrk'
    df_open_orders = pd.concat([df_open_orders_robin,df_open_orders_ibkr],ignore_index=True)
    
    dict_my_portfolio = {"my_current_holdings":df_my_current_holding,
                         "open_orders":df_open_orders,
                         "portfolio_history":df_portfolio_history}
    dict_trade_histroy = {"df_trade_summary":df_trade_summary}
    return({"dict_my_portfolio":dict_my_portfolio,
            "dict_trade_histroy":dict_trade_histroy})

In [ ]:
def holding_summerizer(df,
                       grouby=['sector']):
    df_my_current_holding = df.copy()
    df_summary = df_my_current_holding.groupby(grouby,dropna=False).apply(lambda s: 
                                                        pd.Series({ 
                                                            "ROA": s['ROA'].mean(),
                                                            "ROE": s['ROE'].mean(),
                                                            'percentage': s['percentage'].sum(),
                                                            "equity": s["equity"].sum(),
                                                            "equity_initial": s["equity_initial"].sum(),
                                                            "equity_change": s["equity_change"].sum(),
                                                            'percent_change': np.average(a=s['percent_change'],weights=s['equity_initial']),
                                                            'ev_ebit':s['ev_ebit'].mean(),
                                                            "firm_to_ebit": s['firm_to_ebit'].mean(),
                                                            "pe": s['pe'].mean(),
                                                            "fpe": s['fpe'].astype(float).mean(),
                                                            "pb": s['pb'].mean(),
                                                            "pb_ratio": s['pb_ratio'].mean(),
                                                            "ev": s['ev'].mean(),
                                                            "marketCap": s['marketCap'].mean(),
                                                            "rev": s['rev'].mean(),
                                                            "ebit": s['ebit'].mean(),
                                                            "netIncome": s['netIncome'].mean(),
                                                            "netDebt": s['netDebt'].mean(),
                                                            "grossMargins": s['grossMargins'].mean(),
                                                            "operatingMargins": s['operatingMargins'].mean(),
                                                            "profitMargins": s['profitMargins'].mean(),
                                                            "totalCash": s['totalCash'].mean(),
                                                            "totalDebt": s['totalDebt'].mean(),
                                                            "currentRatio": s['currentRatio'].mean(),
                                                            "quickRatio": s['quickRatio'].mean(),
                                                            "grossProfits": s['grossProfits'].mean(),
                                                            "ebitda": s['ebitda'].mean(),
                                                            "ocf": s['ocf'].mean(),
                                                            "ebitdaMargins": s['ebitdaMargins'].mean(),
                                                            "freeCashflow": s['freeCashflow'].mean(),
                                                            "heldPercentInsiders": s['heldPercentInsiders'].mean(),
                                                            "div_yeild": s['div_yeild'].mean(),
                                                             "5y_div_yeild": s['5y_div_yeild'].mean(),
                                                             "payoutRatio": s['payoutRatio'].mean(),
                                                            "fullTimeEmployees": s['fullTimeEmployees'].mean(),
                                                            'debtToEquity': s['debtToEquity'].mean(),
                                                            #'boker_equity_K': dict(zip(s['broker'],np.round(s['equity']/1000,2)))
                                                        })).sort_values("equity",ascending=False).reset_index()
    return(df_summary)

### Portfolio Stats 

In [ ]:
def portfolio_stats_computer(portfolio_return_series,
                            frequency=252):
    import scipy.stats
    """This Function is to computer Annualized Return and Volatility of a series given the series and its frequency
    VaR and CVaR are at 5%"""
    portfolio_return_series=pd.Series(portfolio_return_series)
    
    portfolio_annualized_compouded_return = (1 + portfolio_return_series).prod() ** (frequency / portfolio_return_series.count()) - 1
    portfolio_annualized_simple_return = portfolio_return_series.mean()*frequency
    portfolio_annualized_volatility= portfolio_return_series.std()*np.sqrt(frequency)
    portfolio_VaR_05, portfolio_CVaR_05 = VaR_CVaR_Quantile(portfolio_return_series,0.05)
    portfolio_sharp_ratio = portfolio_annualized_compouded_return/portfolio_annualized_volatility
    
    portfolio_kurtosis= scipy.stats.kurtosis(portfolio_return_series)+3
    portfolio_skewness= scipy.stats.skew(portfolio_return_series)
    
    dict_stats={"Annualized Simple Return":[portfolio_annualized_simple_return],
                "Annualized Compunded Return":[portfolio_annualized_compouded_return],
                "Annualized Volatility":[portfolio_annualized_volatility],
                "Sharp Ratio":[portfolio_sharp_ratio],
                "Kurtosis":[portfolio_kurtosis],
                "Skewness":[portfolio_skewness],
                "VaR 5":[portfolio_VaR_05],
                "CVaR 5":[portfolio_CVaR_05]}
    return pd.DataFrame(dict_stats)

In [ ]:
def VaR_CVaR_Quantile(data_series,
                      significance_level):
    """Returnn Value at Risk (VaR) and Conditional Value at Risk (CVaR) at significance_level"""
    import scipy.stats
    import numpy as np
    mean=np.mean(data_series)
    std= np.std(data_series)
    ### Data Driven VaR CVaR
    VaR_Quantile = np.quantile(data_series,significance_level)
    return_in_tail_based_on_VaR_Quantile=data_series[data_series<VaR_Quantile]
    CVaR_Quantile=(np.mean(return_in_tail_based_on_VaR_Quantile))
    return(VaR_Quantile,CVaR_Quantile)